In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from main.tooling.Logger import logging_setup

logger = logging_setup("training")

## Check MLflow Connection

In [ ]:
import mlflow
import os
if not os.getenv("MLFLOW_TRACKING_URI"):
   raise Exception("Mlflow not configured")

import requests
try:
    requests.get(str(mlflow.get_tracking_uri()))
except requests.exceptions.ConnectionError:
    raise ConnectionError(f"mlflow not reachable, please run mlflow server")

logger.info("mlflow availiable at %s", mlflow.get_tracking_uri())


## Configurate Experiment Parameters

In [ ]:
experimentName = "RelevanceClassifier"
runName = "AllDatasets"
modelName = "Iteration_1_model"

## Check existing MLflow runs

In [ ]:
from mlflow.entities import ViewType

if mlflow.get_experiment_by_name(experimentName) is None:
        mlflow.create_experiment(experimentName)

mlflow_runs = mlflow.search_runs(
        experiment_names=[experimentName],
        filter_string=f"attributes.status = 'FINISHED' AND params.project.run_name = '{runName}'",                
        max_results=1,
        run_view_type=ViewType.ACTIVE_ONLY,
        order_by=["metrics.accuracy DESC"],        

)

logger.info("mlflow best_run found: %s", not mlflow_runs.empty)


## Model Loading

In [ ]:
from main.tooling.FileManager import getModelPath
from main.tooling.FileManager import cleanup

cleanup()

if not mlflow_runs.empty:
    # Extract run ID
    default_run_id = mlflow_runs.iloc[0]['run_id']
    
    # Load run with the run ID
    default_run = mlflow.get_run(default_run_id)
    
    logger.info("Download Model from mlflow to: %s", getModelPath(""))    

    mlflow.artifacts.download_artifacts(artifact_uri=
        f"{default_run.info.artifact_uri}/{modelName}",
        dst_path=getModelPath(""),
)

else:
    logger.info("The requested model does not exist!")
